## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-09-02-27-50 +0000,nypost,St. James-Smithtown falls game short of LLWS i...,https://nypost.com/2025/08/08/sports/st-james-...
1,2025-08-09-02-15-53 +0000,wapo,"Trump, Putin meeting set for next week in Alaska",https://www.washingtonpost.com/politics/2025/0...
2,2025-08-09-02-13-01 +0000,nypost,Rep. Jasmine Crockett laughs off troubling acc...,https://nypost.com/2025/08/08/us-news/rep-jasm...
3,2025-08-09-02-05-00 +0000,wsj,Beijing Struggles to Shake Off Deflationary Pr...,https://www.wsj.com/economy/beijing-struggles-...
4,2025-08-09-02-02-27 +0000,nypost,Cincinnati viral beating bodycam shows cops at...,https://nypost.com/2025/08/08/us-news/cincinna...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,74
226,gaza,29
374,israel,19
225,plan,16
227,city,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...,184
183,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,171
136,2025-08-08-19-22-00 +0000,wsj,President Trump’s team is reviewing new conten...,https://www.wsj.com/economy/central-banking/tr...,132
198,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan ink histo...,https://nypost.com/2025/08/08/us-news/armenia-...,128
85,2025-08-08-21-32-18 +0000,nypost,Embattled Intel CEO calls out ‘misinformation’...,https://nypost.com/2025/08/08/business/intel-c...,128


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,184,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...
298,108,2025-08-08-03-37-58 +0000,wapo,Israel’s security cabinet approves plan for mi...,https://www.washingtonpost.com/world/2025/08/0...
183,68,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
198,54,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan ink histo...,https://nypost.com/2025/08/08/us-news/armenia-...
55,51,2025-08-08-23-04-25 +0000,cbc,Trump says he'll meet Putin in Alaska next Fri...,https://www.cbc.ca/news/world/trump-putin-meet...
91,41,2025-08-08-21-26-21 +0000,nypost,Officer killed in shooting outside CDC HQ in A...,https://nypost.com/2025/08/08/us-news/active-s...
173,40,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
43,38,2025-08-08-23-49-00 +0000,wsj,President Trump removed former congressman Bil...,https://www.wsj.com/politics/policy/billy-long...
190,37,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...
4,36,2025-08-09-02-02-27 +0000,nypost,Cincinnati viral beating bodycam shows cops at...,https://nypost.com/2025/08/08/us-news/cincinna...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
